# Метод 2 - Использовать Word2Vec модель
#### Будем складывать векторы всех слов

## using [this](http://ling.go.mail.ru/misc/dialogue_2015.html) corpora
### (News corpus and Russian National Corpus)

In [1]:
import requests
import re
import nltk
import pymorphy2
import numpy as np
from html2text import HTML2Text
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from gensim.models.word2vec import Word2Vec

In [2]:
# данные тематики
t = """/Наука и образование
    /Наука и образование/наука
    /Наука и образование/наука/математика
    /Наука и образование/наука/физика
    /Наука и образование/наука/химия
    /Наука и образование/наука/информатика
    /Наука и образование/наука/информатика/биоинформатика
    /Наука и образование/наука/информатика/анализ данных
    /Наука и образование/наука/литература
    /Наука и образование/образование
    /Наука и образование/образование/школьное
    /Наука и образование/образование/высшее
    /Наука и образование/образование/дополнительное
    /Наука и образование/образование/дополнительное/GoTo
    /Политика
    /Политика/Внутренняя
    /Политика/Внешняя
    /Экономика и бизнес
    /Экономика и бизнес/Бизнес
    /Экономика и бизнес/Бизнес/Стартапы
    /Экономика и бизнес/Бизнес/Стартапы/E-Contenta
    /Экономика и бизнес/Бизнес/Крупные компании
    /Экономика и бизнес/Экономика
    /Отдых и развлечения
    /Отдых и развлечения/Кино
    /Отдых и развлечения/Театр
    /Отдых и развлечения/Компьютерные игры
    /Здоровье и красота/Фитнес
    /Здоровье и красота/Медицина
    /Здоровье и красота/Косметология""".split("\n")

# массив последних тем в иерархии (топики) и словарь, с пом. которого можно восстановить полную структуру
topics = [x[1:].split("/")[-1] for x in t]
restore = {topics[i]: t[i] for i in range(len(topics))}

## Описание есть в файле WebClassifier-1.ipynb

In [3]:
web_url = "https://ru.wikipedia.org/wiki/%D0%9E%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B8%D1%82%D0%B5%D0%BB%D1%8C_%D0%92%D0%B0%D0%BD%D0%B4%D0%B5%D1%80%D0%BC%D0%BE%D0%BD%D0%B4%D0%B0"
page = requests.get(web_url)
soup = BeautifulSoup(page.content, "lxml")
h = HTML2Text()
h.ignore_links = True
raw_text = h.handle(soup.body.text)
only_russian_text = re.sub("[^а-яА-Я]", " ", raw_text).strip()
morph = pymorphy2.MorphAnalyzer()
words = only_russian_text.split()
stoplist = stopwords.words('russian')
data = []
for word in words:
    if 2 < len(word) < 16 and word not in stoplist:
        p = morph.parse(word)[0]
        if p.tag.POS == 'NOUN' or "VERB":
            data.append(str(p.normal_form))
count_vect = CountVectorizer()
tf_idf = TfidfTransformer()
X = count_vect.fit_transform([" ".join(data)])
X = tf_idf.fit_transform(X)
tfidf_repr = sorted([(a[0][1], a[1]) for a in X.todok().items()], key=lambda a: a[1], reverse=True)
tfidf_dict = {id_ : tfidf for id_, tfidf in tfidf_repr}

In [4]:
from gensim.models.word2vec import Word2Vec

In [5]:
model = Word2Vec.load_word2vec_format('news.model.bin', binary=True)

In [6]:
a = []
b = []
for k in topics:
    # сначала название топика приведем к нормальной форме, чтобы лучше найти его в count_vect
    word = morph.parse(k)[0].normal_form
    _id = count_vect.vocabulary_.get(word)
    if _id:
        # по слову - айдишник, по айдишнику - tfidf значение. Если оно существует, то добавляем топик и tfidf в соотв. массивы
        tfidf = tfidf_dict.get(_id)
        if tfidf:
            a.append(k)
            b.append(tfidf)

In [7]:
k = 0
# нулевой вектор, он же и результирующий - сложение по правилам numpy
vector_summ = np.zeros(model.vector_size)
for i in range(len(a)):
    try:
        vector_summ += (model[a[i]] * b[i])
    except KeyError:
        k += 1

hundred_similar_words = model.most_similar(positive=[vector_summ], negative=[], topn=100)

In [8]:
hundred_similar_words

[('наука', 0.8318927884101868),
 ('математика', 0.6977936029434204),
 ('литература', 0.6834399700164795),
 ('физика', 0.6183444857597351),
 ('биология', 0.6048916578292847),
 ('естествознание', 0.5761309862136841),
 ('физико-математический', 0.5751566290855408),
 ('информатика', 0.5618166923522949),
 ('лингвистика', 0.5615713596343994),
 ('языкознание', 0.5526806116104126),
 ('культурология', 0.5335828065872192),
 ('филологический', 0.5316666960716248),
 ('обществознание', 0.531338632106781),
 ('химия', 0.5215914249420166),
 ('кибернетика', 0.5192195177078247),
 ('естественнонаучный', 0.517939031124115),
 ('обществоведение', 0.5163986682891846),
 ('литературоведение', 0.5140202641487122),
 ('член-корреспондент', 0.5103638172149658),
 ('астрономия', 0.5091902017593384),
 ('математик', 0.5085898041725159),
 ('алгебра', 0.501548707485199),
 ('педагогика', 0.49928444623947144),
 ('естественно-научный', 0.49542444944381714),
 ('доцент', 0.4946599304676056),
 ('география', 0.4937931299209595

In [9]:
restore[hundred_similar_words[0][0]], hundred_similar_words[0][1]

('    /Наука и образование/наука', 0.8318927884101868)

In [10]:
res = []
try:
    for i in range(len(restore)):
        similar_word = hundred_similar_words[i]
        res.append("{} — {:.2f}%".format(restore[similar_word[0]], similar_word[1] * 100))
except KeyError:
    pass
    
res = sorted(res, key=lambda x: float(x.split(" — ")[1][:-1]), reverse=True)

In [11]:
res

['    /Наука и образование/наука — 83.19%',
 '    /Наука и образование/наука/математика — 69.78%',
 '    /Наука и образование/наука/литература — 68.34%',
 '    /Наука и образование/наука/физика — 61.83%']

## Снова объеденим все блоки и определим класс

In [12]:
import requests
import re
import nltk
import pymorphy2
import numpy as np
from html2text import HTML2Text
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from gensim.models.word2vec import Word2Vec


class Method2():
    def __init__(self):
        t = """/Наука и образование
        /Наука и образование/наука
        /Наука и образование/наука/математика
        /Наука и образование/наука/физика
        /Наука и образование/наука/химия
        /Наука и образование/наука/информатика
        /Наука и образование/наука/информатика/биоинформатика
        /Наука и образование/наука/информатика/анализ данных
        /Наука и образование/наука/литература
        /Наука и образование/образование
        /Наука и образование/образование/школьное
        /Наука и образование/образование/высшее
        /Наука и образование/образование/дополнительное
        /Наука и образование/образование/дополнительное/GoTo
        /Политика
        /Политика/Внутренняя
        /Политика/Внешняя
        /Экономика и бизнес
        /Экономика и бизнес/Бизнес
        /Экономика и бизнес/Бизнес/Стартапы
        /Экономика и бизнес/Бизнес/Стартапы/E-Contenta
        /Экономика и бизнес/Бизнес/Крупные компании
        /Экономика и бизнес/Экономика
        /Отдых и развлечения
        /Отдых и развлечения/Кино
        /Отдых и развлечения/Театр
        /Отдых и развлечения/Компьютерные игры
        /Здоровье и красота/Фитнес
        /Здоровье и красота/Медицина
        /Здоровье и красота/Косметология""".split("\n")
        self.topics = [x[1:].split("/")[-1] for x in t]
        self.restore = {self.topics[i]: t[i] for i in range(len(self.topics))}

        self.morph = pymorphy2.MorphAnalyzer()

        self.model = Word2Vec.load_word2vec_format('ruscorpora.model.bin', binary=True)

    def classify(self, web_url, topn=10):

        # получим текст
        page = requests.get(web_url)
        soup = BeautifulSoup(page.content, "lxml")
        h = HTML2Text()
        h.ignore_links = True
        raw_text = h.handle(soup.body.text)
        only_russian_text = re.sub("[^а-яА-Я]", " ", raw_text).strip()
    
        # оставим только слова без стоп-слов длины [3, 15] и только существительные и глаголы в нормальной форме
    
        words = only_russian_text.split()
        stoplist = stopwords.words('russian')
        data = []
        for word in words:
            if 2 < len(word) < 16 and word not in stoplist:
                p = self.morph.parse(word)[0]
                if p.tag.POS == 'NOUN' or "VERB":
                    data.append(str(p.normal_form))
    
        count_vect = CountVectorizer()
        tf_idf = TfidfTransformer()
        X = count_vect.fit_transform([" ".join(data)])
        X = tf_idf.fit_transform(X)
    
        tfidf_repr = sorted([(a[0][1], a[1]) for a in X.todok().items()], key=lambda a: a[1], reverse=True)
        tfidf_dict = {id_: tfidf for id_, tfidf in tfidf_repr}
    
        a = []
        b = []
        for k in self.topics:
            # сначала название топика приведем к нормальной форме, чтобы лучше найти его в count_vect
            word = self.morph.parse(k)[0].normal_form
            _id = count_vect.vocabulary_.get(word)
            if _id:
                # по слову - айдишник, по айдишнику - tfidf значение. Если оно существует, то добавляем топик и tfidf в соотв. массивы
                tfidf = tfidf_dict.get(_id)
                if tfidf:
                    a.append(k)
                    b.append(tfidf)
    
        # нулевой вектор, он же и результирующий - сложение по правилам numpy
        vector_summ = np.zeros(self.model.vector_size)
        for i in range(len(a)):
            try:
                vector_summ += (self.model[a[i]] * b[i])
            except KeyError:
                pass
    
        hundred_similar_words = self.model.most_similar(positive=[vector_summ], negative=[], topn=1000)
    
        res = []
        try:
            for i in range(len(self.restore)):
                similar_word = hundred_similar_words[i]
                res.append("{} — {:.2f}%".format(self.restore[similar_word[0]], similar_word[1] * 100))
        except KeyError:
            pass
    
        return sorted(res, key=lambda x: float(x.split(" — ")[1][:-1]), reverse=True)[:topn]

In [13]:
m = Method2()
method2 = lambda web_url: m.classify(web_url)

In [14]:
method2("http://goto.msk.ru/hackathon/")

['        /Наука и образование/образование — 91.41%',
 '        /Наука и образование/наука — 58.53%']

In [15]:
method2("https://www.kinopoisk.ru/film/648440/")

[]

In [16]:
method2("https://e-contenta.com/ru/")

[]

In [17]:
method2("https://ru.wikipedia.org/wiki/C%2B%2B")

['        /Наука и образование/наука/математика — 80.89%',
 '        /Наука и образование/наука/литература — 71.03%',
 '        /Наука и образование/наука/информатика — 66.63%',
 '        /Наука и образование/наука/физика — 62.51%']